In [1]:
import pandas as pd
import json
import re
import src.cleaning as cl

In [2]:
df = pd.read_csv("./data/shots_v1.csv")

In [3]:
df.head()

,Unnamed: 0,0,1
0,0,1,"[{'A. Della Valle': ['#local-out', '104.418306..."
1,1,2,"[{'J. Zyskowski': ['#local-in', '24.75', '126...."
2,2,3,"[{'D. DiLeo': ['#local-out', '166.684342478713..."
3,3,4,"[{'J. Rousselle': ['#local-in', '24.75', '134...."
4,4,5,"[{'J. Vaulet': ['#local-dunk', '24.75', '141.2..."


In [4]:
df.rename(columns = {"0": "Jornada"}, inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)

In [5]:
names = cl.shots_per_player(df)

In [6]:
for elem in names:
    if elem == " ":
        names.remove(elem)

In [7]:
len(names)

37285

In [8]:
jornadas, prueba = cl.cumulat_shots(df)


In [9]:
len(jornadas)

33

In [10]:
# As we had problems trying to append the first gameday number of shots, we append it before looping. 
shots_per_jornada = [1157]
for i in range(len(jornadas)-1):
    shots_per_jornada.append(jornadas[i+1]-jornadas[i])

In [11]:
shots_per_jornada

[1157,
 1133,
 1141,
 1133,
 1133,
 1125,
 1135,
 1109,
 1112,
 1101,
 1146,
 1110,
 1107,
 1163,
 1173,
 1138,
 1051,
 1116,
 1140,
 1145,
 1158,
 985,
 1149,
 1157,
 1132,
 1159,
 1128,
 1141,
 1138,
 1147,
 1136,
 1166,
 1121]

In [12]:
num_jornada = list(range(1, 35))
num_jornada = [str(x) for x in num_jornada]

In [13]:
final_jornada = sum([[s] * n for s, n in zip(num_jornada, shots_per_jornada)], [])

In [14]:
#Finally we get a list with values from 1 to 34 (gamedays played) according to when every shot was made. 
len(final_jornada)

37285

### Now we are gonna divide the list "prueba" in order to get data for each three elements. 

### Here we divide the information contained in "prueba" in three lists. One to know if the shot was made or not, as a local or as a visitor. Another for its coordinate x and the last one for its coordinate y.

In [16]:
made = []
for i in range(0, len(prueba), 3):
    made.append(prueba[i])
    
coord_x = []
for i in range(1, len(prueba), 3):
    coord_x.append(prueba[i])
    
coord_y = []
for i in range(2, len(prueba), 3):
    coord_y.append(prueba[i])
    
cositas = list(zip(names, final_jornada, made, coord_x, coord_y))
df = pd.DataFrame(cositas)
df.rename(columns = {0: "name", 1: "jornada", 2: "shot", 3: "coord_x", 4: "coord_y"}, inplace = True)

In [17]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.75,126.41666666666667
1,J. Zyskowski,1,#local-in,24.75,23.883333333333333
2,J. Zyskowski,1,#local-out,25.705345316934725,23.883333333333333
3,J. Zyskowski,1,#local-in,27.616035950804164,137.86666666666667
4,J. Zyskowski,1,#local-in,27.12899716177862,129.81666666666666


In [18]:
df["coord_x"] = pd.to_numeric(df["coord_x"], downcast="float")
df["coord_y"] = pd.to_numeric(df["coord_y"], downcast="float")
df["jornada"] = pd.to_numeric(df["jornada"], downcast="integer")

In [19]:
df_visitor = df.loc[(df["shot"] == "#visitor-in") | (df["shot"] == "#visitor-out") | (df["shot"] == "#visitor-dunk")]
df_local = df.loc[(df["shot"] == "#local-in") | (df["shot"] == "#local-out") | (df["shot"] == "#local-dunk")]
df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)
df = pd.concat([df_local, df_visitor])
df["name"] = df["name"].str.strip()

<ipython-input-19-6cad6d911b17>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)


In [20]:
df.to_csv("./data/database_shots.csv")